# AQME for CSEARH and CMIN module, following which inputs for gaussian is generated

In [ ]:
from rdkit import RDLogger
from rdkit import Chem                                                                                                                                                             
RDLogger.DisableLog('rdApp.*') 

from aqme.csearch import csearch
from aqme.cmin import cmin
from aqme.qprep import qprep
   

#### TASK 1: SMILES input of quinine, CSEARCH (conformer search) with RDKit, create Gaussian input files, CMIN (conformer minimization, additional refinement) with xTB

###### Step 1: doing CSEARCH

In [ ]:
smi = 'COC1=CC2=C(C=CN=C2C=C1)[C@H]([C@@H]3C[C@@H]4CCN3C[C@@H]4C=C)O'
params = 'params.yaml'
name_1 = 'quinine'

In [ ]:
csearch_conf = csearch(smi,name_1,CSEARCH='rdkit')
# run this function using a varfile
# csearch_conf = csearch('SMILES','quinine',varfile=params)

In [ ]:
csearch_data = csearch_conf.compute_confs()
csearch_data

###### Step 2: Doing CMIN with the sdf obtained from CSEARCH

In [ ]:
for file in csearch_conf.csearch_file:
    csearch_mols = Chem.SDMolSupplier(str(file),removeHs=False)
    cmin_conf = cmin(csearch_mols,name_1,CMIN='xtb')
    # run this function using a varfile
    # cmin_conf = cmin(csearch_mols,'quinine',varfile=params)
    

In [ ]:
cmin_data = cmin_conf.compute_cmin()
cmin_data

###### Step 3: Writing Gaussian input files with the sdf obtained from CMIN

In [ ]:
cmin_mols = Chem.SDMolSupplier(str(cmin_conf.cmin_file),removeHs=False)
for i, mol in enumerate(cmin_mols):
    charge= int(mol.GetProp('Real charge'))
    qprep(mol=mol, molecule=f'{name_1}_conf_{i+1}', charge=charge, suffix='CMIN-xTB', qm_input='B3LYP/6-31G** opt freq',mem='24GB',nprocs=8)
    # run this function using a varfile
    # qprep(mol=mol, molecule='quinine_conf_'+str(i+1), charge=charge, varfile=params)
    

#### TASK 2: Metal complex with SMILES input, no suffix, create Gaussian input files with genECP, CSEARCH with RDKit

In [ ]:
smi_metal = 'I[Pd](Cl)([PH3+])[N+]1=CC=CC=C1'
params_metal = 'params-metal.yaml'
name_2 = 'Pd_complex'

In [ ]:
csearch_conf_metal = csearch(smi_metal,name_2,varfile=params_metal)

In [ ]:
csearch_data_metal = csearch_conf_metal.compute_confs()
csearch_data_metal

In [ ]:
for i,file in enumerate(csearch_conf_metal.csearch_file):
    c_mols = Chem.SDMolSupplier(str(file),removeHs=False)
    for j, mol in enumerate(c_mols):
        charge= int(mol.GetProp('Real charge'))
        qprep(mol=mol, molecule=f'{name_2}_{i}_conf_{j+1}',charge=charge, 
            bs_gen='def2svp', bs='6-31G*', gen_atoms=['Pd'],qm_input='B3LYP/genecp opt freq',mem='24GB',nprocs=8)
        # run this function using a varfile
        # qprep(mol=mol, molecule='Pd_conf_'+str(i+1), charge=charge, varfile=params_metal)
 

#### TASK 3: SMILES input of a noncovalent isopentane--water complex, create Gaussian input files, CSEARCH with CREST

In [ ]:
smi = 'CCC(C)C.O'
params = 'params-crest.yaml'

In [ ]:
csearch_conf = csearch(smi,'test',varfile=params)

In [ ]:
csearch_data = csearch_conf.compute_confs()
csearch_data

for file in csearch_conf.csearch_file:
    csearch_mols = Chem.SDMolSupplier(str(file),removeHs=False)
    for i, mol in enumerate(csearch_mols):
        charge= int(mol.GetProp('Real charge'))
        qprep(mol=mol, molecule='ts2_conf_'+str(i+1), charge=charge, suffix='CMIN-xTB', qm_input='B3LYP/6-31G** opt freq',mem='24GB',nprocs=8)

#### TASK 4: TS including a three-component noncovalent complex from SMILES, create ORCA input files, CSEARCH with CREST

In [ ]:
smi = 'O.FC(C)(C)C.[Cl-]'
mol = Chem.MolFromSmiles(smi)
mol = Chem.AddHs(mol)
for i,atom in enumerate(mol.GetAtoms()):
    atom.SetAtomMapNum(i+3) 
smi_new = Chem.MolToSmiles(mol)
print(smi_new)


In [ ]:
mol

In [ ]:
constraits_dist = [[4,5,1.8],[5,9,1.8]]
constraits_angle = [[9,5,4,180]]

In [ ]:
csearch_ts = csearch(smi_new,'ts2',CSEARCH='crest',
                     constraints_dist=constraits_dist,constraints_angle=constraits_angle, ts_complex=True, cregen=True)

In [ ]:
ts_data = csearch_ts.compute_confs()

In [ ]:
ts_data

In [ ]:
for file in csearch_ts.csearch_file:
    csearch_mols = Chem.SDMolSupplier(str(file),removeHs=False)
    for i, mol in enumerate(csearch_mols):
        charge= int(mol.GetProp('Real charge'))
        qprep(mol=mol, molecule='ts2_conf_'+str(i+1), charge=charge, suffix='CMIN-xTB', 
            qm_input='B3LYP def2-SVP opt',program='orca',mem='24GB',nprocs=8)

## For more complex ORCA inputs, just add lines like this:
# ORCA_inp = 'B3LYP def2-SVP opt\n'
# ORCA_inp += '%cpcm\n'
# ORCA_inp += 'smd true\n'
# ORCA_inp += 'SMDsolvent \"CH2Cl2\"\n'
# ORCA_inp += 'end\n'
# and then run qprep with qm_input=ORCA_inp